In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np
import re
from data.make_data import get_links, is_article, save_links

np.random.seed(41)

In [ ]:
# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/bbc-news-articles")

print("Path to dataset files:", path)

Path to dataset files: /home/vscode/.cache/kagglehub/datasets/bhavikjikadara/bbc-news-articles/versions/1


: 

In [ ]:
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "adityakharosekar2/guardian-news-articles",
    'guardian_articles.csv'
)

In [ ]:
df.head()

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
0,us-news/2016/jan/31/iowa-caucus-underdog-candi...,US news,Iowa underdogs put on brave faces despite all ...,https://www.theguardian.com/us-news/2016/jan/3...,As polling day looms and the cameras turn only...,2016-01-31T23:53:37Z,1
1,us-news/2016/jan/31/iowa-caucus-worlds-most-pa...,US news,Iowa caucus: hologram eagle and Jesus star on ...,https://www.theguardian.com/us-news/2016/jan/3...,"In Des Moines on Sunday, the Guardian was give...",2016-01-31T23:46:28Z,2
2,world/2016/jan/31/tanzania-britsh-helicopter-p...,World news,British pilot in Tanzania 'manoeuvred ​to save...,https://www.theguardian.com/world/2016/jan/31/...,A British pilot who was shot dead by an elepha...,2016-01-31T23:43:48Z,3
3,football/2016/jan/31/late-winner-gets-usa-off-...,Football,USA 3-2 Iceland | International friendly match...,https://www.theguardian.com/football/2016/jan/...,USA took a step toward shaking off the ghosts ...,2016-01-31T23:30:49Z,4
4,football/2016/jan/31/blackburn-paul-lambert-ox...,Football,Reinvigorated Paul Lambert reflects after impr...,https://www.theguardian.com/football/2016/jan/...,"The clean-shaven, spectacle free and suspiciou...",2016-01-31T22:30:10Z,5


In [ ]:
df.to_csv('../data/raw/gurdian_articles.csv', index=False)

In [3]:
df = pd.read_csv('../data/raw/gurdian_articles.csv')

In [4]:
n = 1

get_links(df.sample(10).iloc[1,:][['webUrl', 'id']])

{'url': 'https://www.theguardian.com/world/2020/dec/23/lufthansa-fly-fresh-food-uk-shortage-fears-fruit-vegetables-border-french-blockade',
 'id': '114661',
 'links': [{'href': '#img-1', 'link': 'View image in fullscreen'},
  {'href': 'https://www.theguardian.com/world/coronavirus-outbreak',
   'link': 'Coronavirus'},
  {'href': 'https://www.theguardian.com/world/series/coronavirus-live/latest',
   'link': 'Coronavirus – latest updates'},
  {'href': 'https://www.theguardian.com/world/coronavirus-outbreak',
   'link': 'See all our coronavirus coverage'},
  {'href': 'https://www.theguardian.com/profile/sarahbutler',
   'link': 'Sarah Butler'},
  {'href': 'mailto:?subject=Supermarkets airlifting fruit and vegetables to UK amid shortage fears&body=https://www.theguardian.com/world/2020/dec/23/lufthansa-fly-fresh-food-uk-shortage-fears-fruit-vegetables-border-french-blockade?CMP=share_btn_url',
   'link': 'Share'},
  {'href': 'https://www.theguardian.com/world/2020/dec/23/uk-ferry-passenger

In [5]:
df.sample(10).iloc[1,:][['webUrl', 'id']]

webUrl    https://www.theguardian.com/politics/2021/apr/...
id                                                   122562
Name: 122561, dtype: object

In [6]:
res = get_links(df.sample(10).iloc[1,:][['webUrl', 'id']])

for l in res['links']:
    if is_article(l['href']):
        print(l['href'], l['link'])

https://www.theguardian.com/sport/blog Sportblog
https://www.theguardian.com/football/carabao-cup Carabao Cup
https://www.theguardian.com/profile/barneyronay Barney Ronay
https://www.theguardian.com/football/2021/apr/25/manchester-city-tottenham-carabao-cup-match-report Manchester City win fourth Carabao Cup in a row as Laporte sinks SpursRead more
https://www.theguardian.com/football/manchestercity Manchester City
https://www.theguardian.com/football/carabao-cup Carabao Cup
https://www.theguardian.com/football/2021/apr/24/guardiola-warned-citys-hierarchy-over-doomed-european-super-league Guardiola warned Manchester City hierarchy he would speak out over ESLRead more


In [10]:
save_links(res)

## Define a test set

In [45]:
df.sort_values('webPublicationDate',ascending=False,inplace=True)
# Sort to have most recent at the top

In [119]:
n_test = 100

df_test = df.iloc[:n_test]

In [120]:
test_set_links = df_test[['webUrl', 'id']].apply(get_links,axis=1)

In [121]:
def extract_links(blob):
    return list(filter(is_article, [d['href'] for d in blob['links']]))

In [122]:
flat_list = [item for sublist in test_set_links.apply(extract_links) for item in sublist]

In [130]:
flat_list_unique = set(flat_list)

In [123]:
def is_article_in_df(link):
    return link in df['webUrl'].values

In [124]:
is_article_in_df('https://www.theguardian.com/world/2024/jun/20/ukraine-war-russia-accused-of-using-banned-phosphorus-bombs-in-battle-for-bakhmut')

False

In [125]:
is_article_in_df(df.iloc[0]['webUrl'])


True

In [126]:
list(map(is_article_in_df, flat_list[0:10]))

[False, False, False, False, False, False, False, False, True, True]

In [127]:
n_in_db = sum(list(map(is_article_in_df, flat_list_unique)))

In [131]:
def print_status(test_set_links):
    status_string = 'Looking in {:d} articles, {:d} links to articles found, of which {:d} are unique {:.2f}%\n\
    And {:d} are already in DB ({:.2f}%)'\
    .format(test_set_links.shape[0], len(flat_list), len(flat_list_unique), (100 * (len(flat_list_unique) / float(len(flat_list)))),
            n_in_db, (100 * (n_in_db / float(len(flat_list_unique)))))
    print(status_string)


In [132]:
print_status(test_set_links)

Looking in 100 articles, 932 links to articles found, of which 679 are unique 72.85%
    And 342 are already in DB (50.37%)


## Snowball Sample

In [ ]:
test_set_links = df_test[['webUrl', 'id']].apply(get_links,axis=1)

while True:

    flat_list = [item for sublist in test_set_links.apply(extract_links) for item in sublist]

    flat_list_unique = list(set(flat_list))

    is_list_in_db = list(map(is_article_in_df, flat_list_unique))

    n_in_db = sum(is_list_in_db)

    ratio = n_in_db / float(len(flat_list_unique)) 
    
    print_status(test_set_links)
    
    test_set_links = pd.Series(map(get_links,[{'webUrl':url, 'id' : -99999} for url in flat_list_unique]))
    
    # Add these parsed articles into the DB
    df.append(pd.Series(map(get_links,[{'webUrl':url, 'id' : -99999} for url in flat_list_unique])))
    ## Test this

    if ratio > 0.4:
        break

Looking in 679 articles, 932 links to articles found, of which 679 are unique 72.85%
    And 159 are already in DB (23.42%)
Looking in 1954 articles, 3815 links to articles found, of which 1954 are unique 51.22%
    And 289 are already in DB (14.79%)


KeyboardInterrupt: 

In [111]:
print_status(test_set_links)

Looking in 100 articles, 6790 links to articles found, of which 3590 are unique 52.87%
    And 749 are already in DB (20.86%)


In [ ]:
flat_list = [link for link, in_db in zip(flat_list, is_list_in_db) if in_db]

flat_list = list(set(flat_list))